In [ ]:
### packages required
!pip install fair-esm
!pip install torch
!pip install tensorflow
!pip install sklearn

### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [ ]:
def esm_embeddings(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = esm2(batch_tokens, repr_layers=[6], return_contacts=False)
  token_representations = results["representations"][6].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  return embeddings_results


### dataset loading
Notice: you must load the train dataset of the pretrained model (we need it to normalize the new data for prediction after embeddings)

In [ ]:
import numpy as np
import pandas as pd
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()

In [ ]:
# training dataset loading
dataset = pd.read_excel('sample_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('sample_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')


#### Generate embeddings for unknown peptide and the normalize them based on the train dataset

In [ ]:
# test dataset loading
dataset = pd.read_excel('sample_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('sample_test_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')


In [ ]:
# assign the dataset
X_train_data_name = 'sample_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'sample_test_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

#### Predict results based on available model

An example for prediction
```
from keras.models import load_model
model = load_model('content/DPPIV_tensorflow_model')
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
print(predicted_class)
```

In [ ]:
# load GPU for accelerate if available
import tensorflow as tf
tf.config.experimental.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')


In [ ]:
# unzip the model folder
# please change the name to your model zip filename
!unzip model_zip_filename.zip

In [ ]:
# load the model
from keras.models import load_model
model = load_model('content/model_zip_filename')

In [ ]:
# use the pretrained model for prediction
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
print(predicted_class)

##########################################################
Take care, the prediction result (positive and negative) is based on the original dataset. Here, This example only
##########################################################